# Analyzing CIA Factbook Data (Using SQL)
***

## Introduction
In this project, we'll analyse data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/) (2015), a compendium of statistics about all of the countries on Earth.
The Factbook contains demographic information like the following:
- `area` — the total land and water area
- `population` — the global population
- `population_growth` — the annual population growth rate, as a percentage

We'll use SQL in Jupyter Notebook to analyze data from [SQLite factbook database](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db).

In [54]:
# Run this query below if SQL is not installed
# !conda install -yc conda-forge ipython-sql

In [32]:
#connect our Jupyter Notebook to our database file:

%load_ext sql
%sql sqlite:///factbook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @factbook.db'

## Overview of the data

#### Overview of the information on the tables in the database

In [33]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We can see that our database has one table named `facts` and we have information about the name and the type of the colums of the table. Let's now explore in details the `facts` table by displaying its first five rows.

#### Exploration of the table

In [34]:
%%sql
SELECT * 
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:
- `name` — the name of the country
- `area` — the total land and water area
- `area land` — the country's land area in square kilometers
- `area_water` — the country's waterarea in square kilometers.
- `population` — the global population
- `population_growth` — the annual population growth rate, as a percentage
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

## Summary statistics

Let's start by calculating some summary statistics on some colums and look for any outlier countries.

In [35]:
%%sql
SELECT min(population) as min_population,
       max(population) as max_population,
       min(population_growth) as min_population_growth,
       max(population_growth) as max_population_growth
  FROM facts;


 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics above:

- There's a country with a population of `0`
- There's a country with a population of `7256490011` (or more than 7.2 billion people)

Let's find the countries wich are related to these statistics.

In [36]:
%%sql
SELECT name as country, population
  FROM facts
 WHERE population = (SELECT min(population)
                       FROM facts
                    )
       or population = (SELECT max(population)
                          FROM facts
                       );

 * sqlite:///factbook.db
Done.


country,population
Antarctica,0
World,7256490011


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook [page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html):
![Antartica](https://s3.amazonaws.com/dq-content/257/fb_antarctica.png)

We will therefore exclude the row for the whole word during our analysis. Let's recompute the summary statistics we found earlier while excluding the row for the whole world.

In [37]:
%%sql
SELECT min(population) as min_population,
       max(population) as max_population,
       min(population_growth) as min_population_growth,
       max(population_growth) as max_population_growth
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
0,1367485388,0.0,4.02


## Analysis on human population and it's spread

It seems interesting to respond to some questions that will help us to carry out our demographic analysis.

#### Which countries are densily populated?

Let's begin our analysis by finding countries that are densely populated. To do that, we will look at countries that have the highest population density.

In [38]:
%%sql
SELECT name as country, population, area, population / area as population_density
  FROM facts
 ORDER BY population_density DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


country,population,area,population_density
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


It is not really surprising that the countries with the highest population density are islands or states attached to countries, with a narrow superficies.  

It might be more relevant for us to look for the most densely populated countries, identifying them as follows:

- Above-average values for population.
- Below-average values for area.

In [39]:
%%sql
SELECT name as densely_populated_country, population, area, population/area as population_density
  FROM facts
 WHERE population > (SELECT avg(population)
                      FROM facts
                     WHERE name <> 'World')
   AND area < (SELECT avg(area)
                 FROM facts
                WHERE name <> 'World')
 ORDER BY population_density DESC;

 * sqlite:///factbook.db
Done.


densely_populated_country,population,area,population_density
Bangladesh,168957745,148460,1138
"Korea, South",49115196,99720,492
Philippines,100998376,300000,336
Japan,126919659,377915,335
Vietnam,94348835,331210,284
United Kingdom,64088222,243610,263
Germany,80854408,357022,226
Italy,61855120,301340,205
Uganda,37101745,241038,153
Thailand,67976405,513120,132


According to the conditions set out above, we find 14 countries that are densely populated and and among them, the country with the largest population and the highest population density is Bangladesh. This raises the question of the most populous country.

#### Which country has the most people? 
Let's now identify which country has the largest population.

In [40]:
%%sql
SELECT name as most_populous_country, population, population_growth as population_growth_rate, migration_rate
  FROM facts
 WHERE name <> 'World'
 ORDER BY population DESC
 LIMIT 1;                   

 * sqlite:///factbook.db
Done.


most_populous_country,population,population_growth_rate,migration_rate
China,1367485388,0.45,0.44


This query tells us that China is the most populous country in eart.The low rate of population growth in the world's most populous country could be explained by China's longstanding one-child policy to avoid overpopulation. This raises the question of the country with the highest population growth rate.

#### Which country has the highest population growth rate?

Let's now find the country with the highest population growth rate.

In [41]:
%%sql
SELECT name as country_highest_growth_rate, population, population_growth as population_growth_rate, migration_rate
  FROM facts
 ORDER by population_growth DESC
 LIMIT 1;

 * sqlite:///factbook.db
Done.


country_highest_growth_rate,population,population_growth_rate,migration_rate
South Sudan,12042910,4.02,11.47


South Sudan is the country with the highest population growth rate. This may be explained by the high migration rate of the population. According to this [article](https://www.populationdata.net/pays/soudan-du-sud/), South Sudan is a young country, born in 2012, after a civil war, so many refugees return to the country, which could explain the high migration rate and therefore the high population growth rate. This raises the question of which countries' populations will grow the most next year.

#### Which countries will add the most people to their populations next year?

Let's find the countries which will grow the most next year

In [52]:
%%sql
SELECT name as country, population, population_growth, population * population_growth / 100 as population_add_next_year
  FROM facts
 WHERE name <> 'World'
 ORDER BY population_add_next_year DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


country,population,population_growth,population_add_next_year
India,1251695584,1.22,15270686.1248
China,1367485388,0.45,6153684.246
Nigeria,181562056,2.45,4448270.372
Pakistan,199085847,1.46,2906653.3662
Ethiopia,99465819,2.89,2874562.1691
Bangladesh,168957745,1.6,2703323.92
United States,321368864,0.78,2506677.1392
Indonesia,255993674,0.92,2355141.8008000003
"Congo, Democratic Republic of the",79375136,2.45,1944690.8320000002
Philippines,100998376,1.61,1626073.8536


India and China are the world's most populous countries and the countries which will grow the most and according to this [article](https://www.un.org/fr/sections/issues-depth/population/), at this growth rate, India will be more populous than China in a few years. 4 others countries are from Asia and 3 from Africa, according to the same [article](https://www.un.org/fr/sections/issues-depth/population/), this population growth in these countries is largely due to advances in modern medicine and improved living standards, which have led to a significant reduction in mortality. This raises the issue of countries whose population will naturally decline, i.e. where the death rate exceeds the birth rate.

#### Which countries have a higher death rate than birth rate?

Let's identify the countries with higher mortality rate than birth rate.

In [53]:
%%sql
SELECT name as country, birth_rate, death_rate, birth_rate - death_rate as difference_birth_death_rate, population_growth
  FROM facts
 WHERE death_rate > birth_rate
 ORDER BY difference_birth_death_rate
 LIMIT 10;

 * sqlite:///factbook.db
Done.


country,birth_rate,death_rate,difference_birth_death_rate,population_growth
Bulgaria,8.92,14.44,-5.52,0.58
Serbia,9.08,13.66,-4.58,0.46
Latvia,10.0,14.31,-4.3100000000000005,1.06
Lithuania,10.1,14.27,-4.17,1.04
Ukraine,10.72,14.46,-3.74,0.6
Hungary,9.16,12.73,-3.5700000000000003,0.22
Germany,8.47,11.42,-2.9499999999999993,0.17
Slovenia,8.42,11.37,-2.9499999999999993,0.26
Romania,9.14,11.9,-2.76,0.3
Croatia,9.45,12.18,-2.7300000000000004,0.13


In Europe, and in particular in Eastern Europe, deaths overtake births. Fertility rates in Eastern Europe is lower in order to achieve a long-term population renewal threshold.This can be explained by the high youth unemployment and weak economic prospects that discourage people from settling down and start family. In addition, significantly fewer children were born in those countries in the 1990s and 2000s, during the transition from Soviet Union to independant countries, and once people born in these years reach adulthood, their generation will be numerically smaller. Thus, there will simply be fewer parents to give birth in the future. [Read more](https://brilliantmaps.com/former-soviet-union-population/)

## Conclusion

In this project, we have analyzed various demographic statistics for all the countries in the world from the CIA World Factbook 2015. We've come up with many insights. Demographic changes are influenced by the number of births, the death rate and migration. 
The unbridled growth of the world's population comes mainly from Asia and Africa, where advances in modern medicine and improved living standards have significantly reduced mortality rates. 
Europe, on the other hand, is experiencing declining population growth, with fertility rates in European countries and in particular in Easter European countries now below the level needed to achieve long-term population renewal. 

## Acknowledgement

This project is a guided project provided by Dataquest to understand and practice fundamentals in SQL. The project was done in SQLite.